In [13]:
import pandas as pd

# Load the datasets
hotels_df = pd.read_excel('C:/Users/irpay/OneDrive/Documents/GitHub/payam/hotels.xlsx')
guests_df = pd.read_excel('C:/Users/irpay/OneDrive/Documents/GitHub/payam/guests.xlsx')
preferences_df = pd.read_excel('C:/Users/irpay/OneDrive/Documents/GitHub/payam/preferences.xlsx')

# Remove the unnecessary 'Unnamed: 0' columns from all dataframes
hotels_df.drop(columns='Unnamed: 0', inplace=True)
guests_df.drop(columns='Unnamed: 0', inplace=True)
preferences_df.drop(columns='Unnamed: 0', inplace=True)

# Function to implement the customer preference allocation strategy
# Function to implement the customer preference allocation strategy
def customer_preference_allocation(hotels_df, guests_df, preferences_df):
    # Data Integrity Checks
    # Check for Null Values
    if guests_df['guest'].isnull().any():
        raise ValueError("Missing guest data in guests_df")
    if hotels_df['hotel'].isnull().any():
        raise ValueError("Missing hotel data in hotels_df")
    if preferences_df.isnull().any().any():  # Checks for any null value in the entire DataFrame
        raise ValueError("Missing data in preferences_df")

    # Ensure all guests have preferences
    guests_with_preferences = preferences_df['guest'].unique()
    if not all(guest in guests_with_preferences for guest in guests_df['guest']):
        raise ValueError("Not all guests have hotel preferences")

    # Ensure all preferred hotels exist in the hotels list
    hotels_in_preferences = set(preferences_df['hotel'])
    if not hotels_in_preferences.issubset(set(hotels_df['hotel'])):
        raise ValueError("Some hotels in preferences are not in the hotels list")

    # ... rest of your code ...


    # ... rest of your code ...


    # Sort guests by the order of reservation (assuming the order in the guests_df is the reservation order)
    guests_list = guests_df['guest'].tolist()

    # Initialize the result dictionary
    allocation_results = {
        'guest': [],
        'hotel': [],
        'price_paid': [],
        'preference_score': []  # How high the allocated hotel was on the guest's preference list
    }

    # Convert room availability to a dictionary for faster updates
    room_availability = hotels_df.set_index('hotel')['rooms'].to_dict()

    # Pre-compute discounts and room prices for efficiency
    guest_discounts = guests_df.set_index('guest')['discount'].to_dict()
    hotel_prices = hotels_df.set_index('hotel')['price'].to_dict()

    # Iterate over the list of guests
    for guest in guests_list:
        # Get the discount for the guest
        discount = guest_discounts[guest]

        # Get the ordered list of preferred hotels for the guest
        preferred_hotels = preferences_df[preferences_df['guest'] == guest] \
            .sort_values(by='priority')['hotel'].tolist()

        # Try to find an available hotel from the guest's preferences
        for hotel in preferred_hotels:
            if room_availability.get(hotel, 0) > 0:
                # Update room availability
                room_availability[hotel] -= 1

                # Calculate price paid with discount
                room_price = hotel_prices[hotel]
                price_paid = room_price * (1 - discount)

                # Determine preference score (1 is highest preference)
                preference_score = preferred_hotels.index(hotel) + 1

                # Add to results
                allocation_results['guest'].append(guest)
                allocation_results['hotel'].append(hotel)
                allocation_results['price_paid'].append(price_paid)
                allocation_results['preference_score'].append(preference_score)
                break
        else:
            # Guest could not be allocated to any hotel
            allocation_results['guest'].append(guest)
            allocation_results['hotel'].append(None)
            allocation_results['price_paid'].append(0)
            allocation_results['preference_score'].append(None)

    # Convert the results to a DataFrame
    allocation_df = pd.DataFrame(allocation_results)
    return allocation_df

# Apply the customer preference allocation strategy
customer_preference_allocation_df = customer_preference_allocation(hotels_df, guests_df, preferences_df)
customer_preference_allocation_df.head()  # Display the first few allocations


,guest,hotel,price_paid,preference_score
0,guest_1,hotel_168,271.18,1.0
1,guest_2,hotel_369,133.00,1.0
2,guest_3,hotel_157,60.45,1.0
3,guest_4,hotel_235,136.00,1.0
4,guest_5,hotel_121,50.40,1.0


In [8]:
# Run the allocation function
allocation_df = customer_preference_allocation(hotels_df, guests_df, preferences_df)

# Calculating the required metrics
allocated_room_counts = allocation_df['hotel'].value_counts()
hotels_fully_occupied = sum(
    allocated_room_counts.get(hotel, 0) == room_count
    for hotel, room_count in hotels_df.set_index('hotel')['rooms'].items()
)

results = {
    "Guests Accommodated": allocation_df[allocation_df['hotel'].notnull()].shape[0],
    "Average Guest Satisfaction (1 is best)": allocation_df['preference_score'].mean(),
    "Rooms Occupied": allocation_df[allocation_df['hotel'].notnull()].shape[0],  # Same as Guests Accommodated
    "Hotels Occupied": allocation_df['hotel'].nunique(),
    "Hotels Fully Occupied": hotels_fully_occupied,
    "Total Net Earnings": allocation_df['price_paid'].sum(),
    "Average Net Earnings per Hotel": allocation_df.groupby('hotel')['price_paid'].sum().mean()
}

# Convert the results dictionary to a DataFrame for tabular display
results_df = pd.DataFrame(list(results.items()), columns=['Metric', 'Value'])

# Styling
styled_df = results_df.style\
    .background_gradient(cmap='Blues')\
    .set_properties(**{'text-align': 'left', 'font-size': '14pt'})\
    .format({'Value': "{:,.2f}"})\
    .set_table_styles([{'selector': 'th', 'props': [('font-size', '14pt'), ('text-align', 'center')]}])

# Display the styled DataFrame
styled_df


,Metric,Value
0,Guests Accommodated,"3,975.00"
1,Average Guest Satisfaction (1 is best),1.33
2,Rooms Occupied,"3,975.00"
3,Hotels Occupied,400.00
4,Hotels Fully Occupied,255.00
5,Total Net Earnings,"643,597.53"
6,Average Net Earnings per Hotel,"1,608.99"
